In [25]:
from azure.ai.ml import MLClient
from azure.ai.ml import command, Input, Output, load_component
from azure.identity import DefaultAzureCredential
from azure.ai.ml.parallel import parallel_run_function, RunFunction
from azure.ai.ml.constants import AssetTypes, InputOutputModes

In [2]:
sub_id = "b746917e-ceb7-4ae0-81e6-3ccd893cb0de"
rg = "dpv2"
workspace = "dpv2-wks"

ml_client = MLClient(DefaultAzureCredential(), sub_id, rg, workspace)

In [3]:
print(ml_client)

MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x000001C25B8A72E0>,
         subscription_id=b746917e-ceb7-4ae0-81e6-3ccd893cb0de,
         resource_group_name=dpv2,
         workspace_name=dpv2-wks)


In [58]:
prep_data = load_component(path="./src/prepare_data.yml")

In [36]:
print(prep_data)

$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: prepare_file_tabular_data
version: '1'
display_name: Prepare File Tabular Data
type: command
inputs:
  input_data:
    type: uri_folder
outputs:
  file_output_data:
    type: mltable
  tabular_output_data:
    type: mltable
command: python prepare_data.py --input_data ${{inputs.input_data}} --file_output_data
  ${{outputs.file_output_data}} --tabular_output_data ${{outputs.tabular_output_data}}
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:5
code: ./
tags: {}
is_deterministic: true



In [59]:
from azure.ai.ml.entities import Environment

file_batch_inf = parallel_run_function(
    name = "file_batch_score",
    display_name="display: batch score with file dataset",
    description="description: batch score with file dataset",
    # metang: dict is just a way of to create inputs. input has 3 types of signatures. 
    # metang, dict to create the inputs using (one=1, two=2) format
    # metang, below two format are equal
    # method 1
    # inputs={
    # "job_data_path": Input(
    #        type=AssetTypes.MLTABLE,
    #        description="non dict approach create input"
    #    ),
    # }
    # method 2
    # inputs=dict(
    #    job_data_path=Input(
    #        type=AssetTypes.MLTABLE,
    #        description="the data to be split and scored in parallel",
    #    )
    # )
    inputs=dict(  
        job_data_path=Input(
            # metang: Is here file data type also as MLTable. What's the difference between this parallel step and the tabluar batch step below
            type=AssetTypes.MLTABLE,
            description="the data to be split and scored in parallel",
        )
    ),
    outputs=dict(job_output_path=Output(type=AssetTypes.MLTABLE)),
    #outputs={
    #    "job_output_path" : Output(
    #        type = AssetTypes.MLTABLE
    #    )
    #},
    input_data="${{inputs.job_data_path}}",
    instance_count=2,
    max_concurrency_per_instance=1,
    mini_batch_size="1",
    mini_batch_error_threshold=1,
    retry_settings=dict(max_retries=2, timeout=60),
    logging_level="DEBUG",
    task=RunFunction(
        code = "./src",
        # metang: why we use entry_script, instead of like how we define commmand, just use command="python ..."?
        entry_script="file_batch_inference.py",
        program_arguments="--job_output_path ${{outputs.job_output_path}}",
        environment="azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:1",        
    ),
)

In [60]:


tabular_batch_inf = parallel_run_function(
    name = "batch_score_with_tabular_input",
    display_name="display: Batch Score with Tabular Dataset",
    description="description: parallel component for batch score",
    inputs=dict(
        job_data_path=Input(
            type=AssetTypes.MLTABLE,
            description="description: the data to be split and scored",
        ),
        score_model = Input(
            type=AssetTypes.MLTABLE,
            description="description: input model"
        ),
    ),
    outputs=dict(job_output_path=Output(type=AssetTypes.MLTABLE)),
    # metang: is input_data a fixed input
    input_data="${{inputs.job_data_path}}", # i was using input_data_path, and caused ""evalue": "(UserError) Value cannot be null. "
    instance_count=2,
    max_concurrency_per_instance=2,
    mini_batch_size="100",
    mini_batch_error_threshold=5,
    logging_level="DEBUG",
    retry_settings=dict(max_retries=2, timeout=60),
    task=RunFunction(
        code = "./src",
        entry_script="tabula_batch_inference.py",
        environment=Environment(
            image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
            conda_file="./src/environment_parallel.yml",
        ),
        append_row_to="${{outputs.job_output_path}}",
        program_arguments="--model ${{inputs.score_model}} "
        "--job_output_path ${{outputs.job_output_path}} "
        # metang: not understand why we need use --error_threshold 5 here again. why do we need define mini_batch_error_threshold=5 in code 4 lines above, not in program_argumentics here.
        "--error_threshold 5 "
        "--allowed_failed_percent 30 "
        "--task_overhead_timeout 1200 "
        "--progress_update_timeout 600 "
        "--first_task_creation_timeout 600 "
        "--copy_logs_to_parent True "
        "--resource_monitor_interva 20 "  
    ),
)

In [61]:
from azure.ai.ml.dsl import pipeline

@pipeline()
def parallel_in_pipeline(job_data_path, score_model_path):

    # step 1: prep_data
    prep_data_step = prep_data(input_data=job_data_path)
    # metang: why we need define the output type again. This is already defined in the component yaml?
    prep_data_step.outputs.file_output_data.type = AssetTypes.MLTABLE
    prep_data_step.outputs.tabular_output_data.type = AssetTypes.MLTABLE

    # step 2: batch process data (prs)
    batch_inf_file_step = file_batch_inf(
        job_data_path = prep_data_step.outputs.file_output_data
    )
    # metang: it's interesting to see that define the input type is not in the prs constructor parallel_run_function. That function expose the input, but the actual input needs to be defined the type in the pipeline constructor
    batch_inf_file_step.inputs.job_data_path.mode = (
        InputOutputModes.EVAL_MOUNT
    )
    batch_inf_file_step.outputs.job_output_path.type = AssetTypes.MLTABLE

    # step 3: batcn inferencing (prs)
    batch_inf_tabular_step = tabular_batch_inf(
        job_data_path = prep_data_step.outputs.tabular_output_data,
        score_model=score_model_path,        
    )
    # metang: direct mode mean using data asset id, isn't it?
    batch_inf_tabular_step.inputs.job_data_path.mode = (
        InputOutputModes.DIRECT
    )
    
    return {
        "job_output_file": batch_inf_file_step.outputs.job_output_path,
        "job_output_tabular": batch_inf_tabular_step.outputs.job_output_path,
    }

In [62]:
job_data_path = Input(
    path="./dataset/", type=AssetTypes.MLTABLE, mode=InputOutputModes.RO_MOUNT
)
score_model_path = Input(
    path="./model/", type=AssetTypes.URI_FOLDER, mode=InputOutputModes.DOWNLOAD
)

In [63]:


job = parallel_in_pipeline(job_data_path=job_data_path, score_model_path=score_model_path)


In [64]:
# metang: why here we dond define job.outputs.job_output_file type, but only define job.outputs.job_output_tabular??
job.outputs.job_output_tabular.type = AssetTypes.URI_FILE

job.settings.default_compute = "cpu-cluster"

In [66]:
ret_job = ml_client.jobs.create_or_update(job, experiment_name="prs_demo")



In [ ]:
ret_job.services["Studio"].endpoint

In [53]:
# load component
prepare_data = load_component(path="./src/prepare_data.yml")

In [54]:
# parallel task to process file data
file_batch_inference = parallel_run_function(
    name="file_batch_score",
    display_name="Batch Score with File Dataset",
    description="parallel component for batch score",
    inputs=dict(
        job_data_path=Input(
            type=AssetTypes.MLTABLE,
            description="The data to be split and scored in parallel",
        )
    ),
    outputs=dict(job_output_path=Output(type=AssetTypes.MLTABLE)),
    input_data="${{inputs.job_data_path}}",
    instance_count=2,
    max_concurrency_per_instance=1,
    mini_batch_size="1",
    mini_batch_error_threshold=1,
    retry_settings=dict(max_retries=2, timeout=60),
    logging_level="DEBUG",
    task=RunFunction(
        code="./src",
        entry_script="file_batch_inference.py",
        program_arguments="--job_output_path ${{outputs.job_output_path}}",
        environment="azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:1",
    ),
)

In [55]:
# parallel task to process tabular data
tabular_batch_inference = parallel_run_function(
    name="batch_score_with_tabular_input",
    display_name="Batch Score with Tabular Dataset",
    description="parallel component for batch score",
    inputs=dict(
        job_data_path=Input(
            type=AssetTypes.MLTABLE,
            description="The data to be split and scored in parallel",
        ),
        score_model=Input(
            type=AssetTypes.URI_FOLDER, description="The model for batch score."
        ),
    ),
    outputs=dict(job_output_path=Output(type=AssetTypes.MLTABLE)),
    input_data="${{inputs.job_data_path}}",
    instance_count=2,
    max_concurrency_per_instance=2,
    mini_batch_size="100",
    mini_batch_error_threshold=5,
    logging_level="DEBUG",
    retry_settings=dict(max_retries=2, timeout=60),
    task=RunFunction(
        code="./src",
        entry_script="tabular_batch_inference.py",
        environment=Environment(
            image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
            conda_file="./src/environment_parallel.yml",
        ),
        program_arguments="--model ${{inputs.score_model}} "
        "--job_output_path ${{outputs.job_output_path}} "
        "--error_threshold 5 "
        "--allowed_failed_percent 30 "
        "--task_overhead_timeout 1200 "
        "--progress_update_timeout 600 "
        "--first_task_creation_timeout 600 "
        "--copy_logs_to_parent True "
        "--resource_monitor_interva 20 ",
        append_row_to="${{outputs.job_output_path}}",
    ),
)

In [56]:
@pipeline()
def parallel_in_pipeline(pipeline_job_data_path, pipeline_score_model):

    prepare_file_tabular_data = prepare_data(input_data=pipeline_job_data_path)
    # output of file & tabular data should be type MLTable
    prepare_file_tabular_data.outputs.file_output_data.type = AssetTypes.MLTABLE
    prepare_file_tabular_data.outputs.tabular_output_data.type = AssetTypes.MLTABLE

    batch_inference_with_file_data = file_batch_inference(
        job_data_path=prepare_file_tabular_data.outputs.file_output_data
    )
    # use eval_mount mode to handle file data
    batch_inference_with_file_data.inputs.job_data_path.mode = (
        InputOutputModes.EVAL_MOUNT
    )
    batch_inference_with_file_data.outputs.job_output_path.type = AssetTypes.MLTABLE

    batch_inference_with_tabular_data = tabular_batch_inference(
        job_data_path=prepare_file_tabular_data.outputs.tabular_output_data,
        score_model=pipeline_score_model,
    )
    # use direct mode to handle tabular data
    batch_inference_with_tabular_data.inputs.job_data_path.mode = (
        InputOutputModes.DIRECT
    )

    return {
        "pipeline_job_out_file": batch_inference_with_file_data.outputs.job_output_path,
        "pipeline_job_out_tabular": batch_inference_with_tabular_data.outputs.job_output_path,
    }


pipeline_job_data_path = Input(
    path="./dataset/", type=AssetTypes.MLTABLE, mode=InputOutputModes.RO_MOUNT
)
pipeline_score_model = Input(
    path="./model/", type=AssetTypes.URI_FOLDER, mode=InputOutputModes.DOWNLOAD
)
# create a pipeline
pipeline_job = parallel_in_pipeline(
    pipeline_job_data_path=pipeline_job_data_path,
    pipeline_score_model=pipeline_score_model,
)
pipeline_job.outputs.pipeline_job_out_tabular.type = AssetTypes.URI_FILE

# set pipeline level compute
pipeline_job.settings.default_compute = "cpu-cluster"

In [57]:
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_samples"
)
pipeline_job

Uploading dataset (0.09 MBs): 100%|##########| 85267/85267 [00:01<00:00, 43272.28it/s]


Uploading model (0.0 MBs): 100%|##########| 2227/2227 [00:01<00:00, 1781.52it/s]




Experiment,Name,Type,Status,Details Page
pipeline_samples,gentle_pasta_2xmkwcg7zh,pipeline,Preparing,Link to Azure Machine Learning studio
